In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt


twitch_df = pd.read_csv("Twitch_game_data.csv", encoding='latin-1')
igdb_df = pd.read_csv("data.csv", encoding='UTF-8')

twitch_df.columns = twitch_df.columns.str.strip()
igdb_df.columns = igdb_df.columns.str.strip()

print("Twitch Data Columns:", twitch_df.columns)
print("IGDB Data Columns:", igdb_df.columns)

merged_df = pd.merge(twitch_df, igdb_df, on='Game', how='left')
print("merged_df：", merged_df.columns)

for col in ['Rank', 'Hours_watched', 'Month', 'Year', 'Avg_viewers', 'Hours_streamed', 'Peak_viewers', 'Streamers']:
    if col + "_x" in merged_df.columns and col + "_y" in merged_df.columns:
        merged_df[col] = merged_df[col + "_x"].fillna(merged_df[col + "_y"])
        merged_df.drop(columns=[col + "_x", col + "_y"], inplace=True)
    elif col + "_x" in merged_df.columns:
        merged_df.rename(columns={col + "_x": col}, inplace=True)
    elif col + "_y" in merged_df.columns:
        merged_df.rename(columns={col + "_y": col}, inplace=True)

if "Rank" not in merged_df.columns or "Hours_watched" not in merged_df.columns:
    raise KeyError("lack 'Rank' or 'Hours_watched' ，check dateset。")

if "Release_Year" in merged_df.columns:
    merged_df["Release_Year"] = pd.to_numeric(merged_df["Release_Year"], errors="coerce")
    merged_df.dropna(subset=["Release_Year"], inplace=True)
    merged_df["Release_Year"] = merged_df["Release_Year"].astype(int)
else:
    raise KeyError("lack 'Release_Year' ")

selected_features = ['Hours_watched', 'Month', 'Year']

print("missing value：\n", merged_df[selected_features].isna().sum())

merged_df['Rank_log'] = np.log1p(merged_df['Rank'])

features = selected_features  
target = 'Rank_log'

X = merged_df[features]
y = merged_df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

mlr_model = LinearRegression()
mlr_model.fit(X_train_scaled, y_train)

y_pred_log = mlr_model.predict(X_test_scaled)
y_pred = np.expm1(y_pred_log)      
y_test_actual = np.expm1(y_test)

mse = mean_squared_error(y_test_actual, y_pred)
r2 = r2_score(y_test_actual, y_pred)

print("\nMLR result：")
print(f"RMSE: {np.sqrt(mse):.2f}")
print(f"R²: {r2:.4f}")


future_year = 2025
future_month = 6

X_future = merged_df[(merged_df['Year'] <= future_year) & (merged_df['Month'] <= future_month)]

X_future_games = X_future[['Game']].copy()

X_future = X_future.drop(columns=['Rank', 'Game', 'Rank_log'], errors='ignore')

missing_cols = set(features) - set(X_future.columns)
for col in missing_cols:
    X_future[col] = 0

X_future = X_future[features]

X_future_scaled = scaler.transform(X_future)

predictions_log = mlr_model.predict(X_future_scaled)
predictions = np.expm1(predictions_log)
X_future['Predicted_Rank'] = predictions

X_future['Game'] = X_future_games['Game'].values

X_future = X_future.sort_values('Predicted_Rank').drop_duplicates(subset=['Game'])

top_10_games = X_future.head(10)
top_50_games = X_future.head(50)
top_100_games = X_future.head(100)
top_200_games = X_future.head(200)

print(f"\n**Top 10 Games for {future_year}-{future_month}**")
print(top_10_games[['Game', 'Predicted_Rank']])

print(f"\n**Top 50 Games for {future_year}-{future_month}**")
print(top_50_games[['Game', 'Predicted_Rank']])

print(f"\n**Top 100 Games for {future_year}-{future_month}**")
print(top_100_games[['Game', 'Predicted_Rank']])

print(f"\n**Top 200 Games for {future_year}-{future_month}**")
print(top_200_games[['Game', 'Predicted_Rank']])


Twitch Data Columns: Index(['Rank', 'Game', 'Month', 'Year', 'Hours_watched', 'Hours_streamed',
       'Peak_viewers', 'Peak_channels', 'Streamers', 'Avg_viewers',
       'Avg_channels', 'Avg_viewer_ratio'],
      dtype='object')
IGDB Data Columns: Index(['Rank', 'Game', 'Month', 'Year', 'Hours_watched', 'Hours_streamed',
       'Peak_viewers', 'Peak_channels', 'Streamers', 'Avg_viewers',
       'Avg_channels', 'Avg_viewer_ratio', 'Genre', 'Release_Year',
       'Release_Month', 'New_Game', 'Wrong', 'rank_category'],
      dtype='object')
merged_df： Index(['Rank_x', 'Game', 'Month_x', 'Year_x', 'Hours_watched_x',
       'Hours_streamed_x', 'Peak_viewers_x', 'Peak_channels_x', 'Streamers_x',
       'Avg_viewers_x', 'Avg_channels_x', 'Avg_viewer_ratio_x', 'Rank_y',
       'Month_y', 'Year_y', 'Hours_watched_y', 'Hours_streamed_y',
       'Peak_viewers_y', 'Peak_channels_y', 'Streamers_y', 'Avg_viewers_y',
       'Avg_channels_y', 'Avg_viewer_ratio_y', 'Genre', 'Release_Year',
       'Rel